In [1]:
# Ставим необходимые библиотеки
!pip install requests beautifulsoup4 

In [1]:
# Нужные библиотеки для проекта
import requests
from bs4 import BeautifulSoup 

import pandas as pd

import re
import time
import json

In [2]:
# Загрузим существующий набор данных, собранный с сайта auto.ru 
test_df = pd.read_csv('test.csv')
test_df.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [3]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [5]:
# Список брендов, которые ранее использовались для сбора набора данных.
brands = [brand.lower() for brand in test_df.brand.unique()]
brands

['skoda',
 'audi',
 'honda',
 'volvo',
 'bmw',
 'nissan',
 'infiniti',
 'mercedes',
 'toyota',
 'lexus',
 'volkswagen',
 'mitsubishi']

Для каждого бренда, который будет использоваться для анализа (12 брендов), мы хотим извлечь ссылки на все объявления автомобилей в файле. Этот файл будет использован на следующем этапе сбора данных.

In [6]:
def collect_car_urls(
    brand: str
):
    """
    Собирем все URL автомобилей указанной марки с сайта `auto.ru`.
    Для каждого бренда рассчитывается количество доступных страниц и сохраняются URL-адреса всех этих страниц.
    в список.
    """
    main_url = f'https://auto.ru/cars/{brand}/all/'   
    main_response = requests.get(main_url)   
    main_soap = BeautifulSoup(main_response.content.decode('utf-8'), 'html.parser')    
    _ = main_soap.find('span', class_='ButtonWithLoader__content').text.replace(u'\xa0', '')
    urls_total = int(re.findall(r'\d+', _)[0])
    ads_per_page = len(main_soap.find_all('a', class_='Link ListingItemTitle__link'))
    pages_num = urls_total // ads_per_page
    
    all_urls = []
    
    for page_num in range(1, pages_num):
        if page_num % 20 == 0:
            print(f"Extracting page {page_num} from {pages_num}...")
        page_url = f'{main_url}?page={page_num}'   
        page_response = requests.get(page_url)
        time.sleep(0.1)
        page_soap = BeautifulSoup(page_response.content.decode('utf-8'), 'html.parser')   
        
        all_urls.extend([a.get('href') for a in page_soap.find_all('a', class_='Link ListingItemTitle__link')])

    return all_urls

In [ ]:
urls = []
for brand in brands:
    print(f"Extracting data for the brand {brand}:")
    all_urls = collect_car_urls(brand)
    urls.extend(all_urls)

Extracting data for the brand skoda:


In [ ]:
train_df = pd.DataFrame({'car_url': urls})
train_df.to_csv('train_df.csv', index=False)

In [ ]:
train_df.info()

Для всех сохраненных URL-адресов нам нужно извлечь ту же информацию, которая уже сохранена в нашем тестовом наборе данных.

In [ ]:
extracted_columns = test_df.columns.to_list()
extracted_columns

В тестовых данных 32 функции, за исключением двух столбцов: информация о модели и поставщик, информация которого не является информативной. Возможно модель улутшит добавление еще несколько столбцов:

-'views'      - сколько раз было просмотрено объявление.
-'date_added' - дата, когда объявление было размещено на портале
-'region'     - регион, в котором находится машина
-'price'      - целевой столбец

In [ ]:
def extract_url_data(url: str, extracted_columns: list):
    """
    Для указанного url извлекаем информацию со страницы `auto.ru` обо всех необходимых функциях с указанием
    в списке extracted_columns, например model_name, mileage, price и т. д.
    Функция возвращает список значений для всех указанных функций в том же порядке, в котором указаны ее поля.
    представлен в списке extracted_columns.
    """
    response = requests.get(url)
    page = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
    
    try:
        catalog_url = page.find(
            'a', class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default').get('href')
        response_catalog = requests.get(catalog_url)
        page_catalog = BeautifulSoup(response_catalog.content.decode('utf-8'), 'html.parser')
    except:
        pass
    try:
        json_data_catalog = json.loads(
            page_catalog.find('script', type="application/json", id='initial-state').string)
    except:
        pass
    try:
        json_data_equip = json.loads(
            page.find('script', type="application/json", id='initial-state').string)
    except:
        pass
    try:
        data = json.loads(
            page.find('script', type="application/ld+json").string)
        data = flatten(data)
    except:
        pass
    
    try:
        data['complectation_dict'] = [
        k for k, v in json_data_catalog['state']['compare']['selected'][0]['options'].items() if v == 1]
    except:
        pass
    try:
        data['equipment_dict'] = json_data_equip['card']['vehicle_info']['equipment']
    except:
        pass
    try:
        mileage = page.find(
            'li', class_='CardInfoRow CardInfoRow_kmAge').find_all('span')[-1].text.replace(u'\xa0', u'')
        data['mileage'] = int(re.findall(r'\d+', mileage)[0])
    except:
        pass
    try:
        data['model_name'] = page.find_all(
        'div', class_='InfoPopup InfoPopup_theme_plain InfoPopup_withChildren BreadcrumbsPopup')[1].text
    except:
        pass
    try:
        data['parsing_unixtime'] = int(time.time())
    except:
        pass
    try:
        data['sell_id'] = int(re.findall(
            r'\d+', page.find('div', class_='CardHead__infoItem CardHead__id').text)[0])
    except:
        pass
    try:
        data['super_gen'] = json.loads(
            page.find('div', id="sale-data-attributes").get('data-bem'))
    except:
        pass
    try:
        data['Владельцы'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_ownersCount').find_all('span')[-1].text.replace(u'\xa0', u' ')
    except:
        pass
    try:
        data['Владение'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_owningTime').find_all('span')[-1].text
    except:
        pass
    try:
        data['ПТС'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[-1].text
    except:
        pass
    try:
        data['Привод'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[-1].text
    except:
        pass
    try:
        data['Руль'] = page.find('li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[-1].text
    except:
        pass
    try:
        data['Состояние'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_state').find_all('span')[-1].text
    except:
        pass
    try:
        data['Таможня'] = page.find(
            'li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[-1].text
    except:
        pass
    try:
        data['description'] = re.sub('\W+', ' ', data['description'])
    except:
        pass
    
    # Дополнение
    try:
        data['views'] =  page.find(
            'div', class_='CardHead__infoItem CardHead__views').text.split()[0]
    except:
        pass
    try:
        data['date_added'] = page.find(
            'div', class_='CardHead__infoItem CardHead__creationDate').text 
    except:
        pass
    try:
        data['region'] = page.find(
            'div', class_='CardBreadcrumbs').find_all(
            'div', class_='CardBreadcrumbs__item')[-1].text.replace(u'\xa0', u' ')
    except:
        pass
    
    output = []
    try:
        for col in extracted_columns:
            output.append(data.get(col, None))    
    except:
        pass
    if not output:
        output = [None] * len(extracted_columns)
    return output

In [ ]:
df_combined = pd.read_csv('train_df.csv')
df_combined.info()

In [ ]:
len(df_combined.car_url.values.tolist())

In [ ]:
final_list = []
for n, url in enumerate(df_combined.car_url.values.tolist()[79201:]):
    if n % 50 == 0:
        print(f"URL-адреса обработки {n}, URL {url}.")
    if n % 1000 == 0:
        print(f"Дополняем уже собранными данными.")
        df1 = pd.DataFrame(data=final_list, columns=extracted_columns + ['views', 'date_added', 'region', 'price'])
        final_df = pd.concat([final_df, df1], ignore_index=True)
        final_df.to_csv('train_df_full_part1.csv', index=False)
        final_list = []
    final_list.append(extract_url_data(url, extracted_columns + ['views', 'date_added', 'region', 'price']))

In [ ]:
final_df.info()

In [ ]:
final_df.head()